In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow import keras

def swish_activation(x):
  return (K.sigmoid(x) * x)

class FixedDropout(tf.keras.layers.Dropout):
  def _get_noise_shape(self, inputs):
    if self.noise_shape is None:
      return self.noise_shape

    symbolic_shape = K.shape(inputs)
    noise_shape = [symbolic_shape[axis] if shape is None else shape
                    for axis, shape in enumerate(self.noise_shape)]
    return tuple(noise_shape)

In [3]:
@tf.keras.utils.register_keras_serializable()
class wBiFPNAdd(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-4, **kwargs):
        super(wBiFPNAdd, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        num_in = len(input_shape)
        self.w = self.add_weight(name=self.name,
                                 shape=(num_in,),
                                 initializer=keras.initializers.constant(1 / num_in),
                                 trainable=True,
                                 dtype=tf.float32)

    def call(self, inputs, **kwargs):
        w = keras.activations.relu(self.w)
        # print(w)
        # print(inputs)
        x = tf.reduce_sum([w[i] * inputs[i] for i in range(len(inputs))], axis=0)
        x = x / (tf.reduce_sum(w) + self.epsilon)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape[0]

    def get_config(self):
        config = super(wBiFPNAdd, self).get_config()
        config.update({
            'epsilon': self.epsilon
        })
        return config

In [5]:
class BatchNormalization(tf.keras.layers.BatchNormalization):
  def __init__(self, **kwargs):
    if not kwargs.get('name', None):
      kwargs['name'] = 'tpu_batch_normalization'
    super().__init__(**kwargs)

  def call(self, inputs, training=None):
    outputs = super().call(inputs, training)
    for u in self.updates:
      tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, u)
    return outputs
  
def batch_norm_class(is_training, strategy=None):
    return BatchNormalization
  
def build_batch_norm(is_training_bn,
                     beta_initializer='zeros',
                     gamma_initializer='ones',
                     data_format='channels_last',
                     momentum=0.99,
                     epsilon=1e-3,
                     strategy=None,
                     name='tpu_batch_normalization'):
  axis = 1 if data_format == 'channels_first' else -1
  batch_norm_class_ = batch_norm_class(is_training_bn, strategy)

  bn_layer = batch_norm_class_(
      axis=axis,
      momentum=momentum,
      epsilon=epsilon,
      center=True,
      scale=True,
      beta_initializer=beta_initializer,
      gamma_initializer=gamma_initializer,
      name=name)

  return bn_layer

In [6]:
@tf.keras.utils.register_keras_serializable()
class SegmentationHead(tf.keras.layers.Layer):
  def __init__(self,
               num_classes,
               num_filters,
               min_level,
               max_level,
               data_format,
               is_training_bn,
               act_type,
               strategy,
               head_strategy='transpose',
               upsampling_last=False,
               **kwargs):

    super(SegmentationHead, self).__init__(**kwargs)

    self.num_classes = num_classes
    self.num_filters = num_filters
    self.min_level = min_level
    self.max_level = max_level
    self.data_format = data_format
    self.is_training_bn = is_training_bn
    self.act_type = act_type
    self.strategy = strategy
    self.head_strategy = head_strategy
    self.upsampling_last = upsampling_last

    self.act_type = act_type
    self.con2d_ts = []
    self.con2d_t_bns = []
    max_level += 1  # chg 20201205 add just like unet
    ks = 3
    head_stride = 2
    for i in range(max_level - min_level):
      self.con2d_ts.append(
          tf.keras.layers.Conv2DTranspose(
              num_filters,
              ks,
              strides=2,
              padding='same',
              data_format=data_format,
              use_bias=False,
              name=f'cond2d_ts_{i}'))
      self.con2d_t_bns.append(
          build_batch_norm(
              is_training_bn=is_training_bn,
              data_format=data_format,
              strategy=strategy,
              name=f'bn_{i}'))

    # TODO 封装一个upsampling 传递参数 upsampling,resize,dconv/transpose
    self.head_process = None
    head_strategy = self.head_strategy
    if head_strategy == 'transpose':
        self.head_process  = tf.keras.layers.Conv2DTranspose(
            num_classes, ks, strides=head_stride, padding='same', name='head_transpose')
        self.head_conv = None
    elif head_strategy == 'upsampling':
        scale_size = 4
        self.head_process = tf.keras.layers.UpSampling2D(size=scale_size, interpolation='bilinear')
        self.head_conv = tf.keras.layers.Conv2D(num_classes, kernel_size=(1,1), padding='same')
    elif head_strategy == 'resize':
        image_size = gezi.get('image_size')
        assert image_size
        self.head_process = tf.keras.layers.Lambda(lambda x: mt.image.resize(x, image_size))
        self.head_conv = tf.keras.layers.Conv2D(num_classes, kernel_size=(1,1), padding='same')
    else:
        raise ValueError(head_strategy)

#     dropout = gezi.get('effdet_dropout', flags=True)
    dropout = 0
    if dropout:
        self.dropout = tf.keras.layers.Dropout(float(dropout))
    else:
        self.dropout = None

  # NotImplementedError: Layer SegmentationHead has arguments in `__init__` and therefore must override `get_config`. 如果没有get_config
  def get_config(self):
    config = {
        'num_classes': self.num_classes,
        'num_filters': self.num_filters,
        'min_level': self.min_level,
        'max_level': self.max_level,
        'data_format': self.data_format,
        'is_training_bn': self.is_training_bn,
        'act_type': self.act_type,
        'strategy': self.strategy,
        'head_strategy': self.head_strategy,
        'upsampling_last': self.upsampling_last,
        }
    base_config = super(SegmentationHead, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

  def call(self, feats, training):
    # print('---------', len(feats))
    # print([(x.shape, x.name) for x in feats])
    # print(feats)
    x = feats[-1] # TODO dropout here like unet ?

    if self.dropout is not None:
        x = self.dropout(x)

    skips = list(reversed(feats[:-1]))
    # print([(x.shape, x.name) for x in skips])

    if len(skips) < len(self.con2d_t_bns):
      skips.append(None)

    for con2d_t, con2d_t_bn, skip in zip(self.con2d_ts, self.con2d_t_bns,
                                         skips):
      #  print('-----input', x.shape, skip.shape if skip is not None else None)
      if skip is not None and x.shape[1] * 2 != skip.shape[1]:
        x = mt.image.resize(x, skip.shape[1:3])
      else:
        x = con2d_t(x)
      x = con2d_t_bn(x, training)
#       from  gseg.third.automl.efficientdet import utils
      ## TODO下面写法save graph有点问题 其实最终实际等价标准tf.nn.swish
      #   x = utils.activation_fn(x, self.act_type)
      x = tf.keras.layers.Activation(self.act_type)(x)
      if skip is not None:
        x = tf.concat([x, skip], axis=-1)
    #   print('------output', x.shape)

    # This is the last layer of the model
    # 如果是transpose注意 最后到128 后面还需要resize一次，如果是upsampling或者resize 这里直接resize到输出大小 *4

    notop = x
    if self.head_conv is not None and not self.upsampling_last:
      x = self.head_conv(x)

    x = self.head_process(x)  # 64x64 -> 128x128

    if self.head_conv is not None and self.upsampling_last:
      x = self.head_conv(x)

    # print('------------final shape', x.shape)
    return x, notop

In [7]:
model_path = '../working/v6/effdet/bak/model.h5'
custom_objects = {
                  'tf': tf, 
                  'swish_activation': swish_activation,
                  'FixedDropout': FixedDropout,
                  'wBiFPNAdd': wBiFPNAdd,
                  'SegmentationHead': SegmentationHead
                  }
import pickle
m = pickle.load(open(model_path, 'rb'))
model = tf.keras.models.model_from_json(m['net'], custom_objects=custom_objects)
model.set_weights(m['weights'])

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [8]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [9]:
from IPython.display import display
import os, glob
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import gezi
from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import melt as mt
from gseg.dataset import Dataset
from gseg.loss import get_loss_fn
from gseg.metrics import get_metrics
from gseg import util
from gseg.util import *
gezi.set_pandas()

In [10]:
CLASSES = ['water', 'track_road', 'build', 'track_airport', 'other_park', 'other_playground', 'arable_natural', 'arable_greenhouse',
           'grass_natural', 'grass_greenbelt', 'forest_natural', 'forest_planted', 'bare_natural', 'bare_planted', 'other_other']
NUM_CLASSES = len(CLASSES)

In [11]:
mt.init_flags()
FLAGS = mt.get_flags()
FLAGS.batch_parse = False
batch_size = 32
batch_size = 16
# batch_size = 8
mt.set_global('batch_size', batch_size) # loss fn used / mt.batch_size()
eval_files = gezi.list_files('../input/quarter/tfrecords/train/1/*')
# eval_files = gezi.list_files('../input/quarter/tfrecords/train/1/21.*')
print(eval_files)
eval_dataset = Dataset('valid').make_batch(batch_size, eval_files)
train_files = gezi.list_files('../input/quarter/tfrecords/train/*/*')
train_files = [x for x in train_files if not x in eval_files]
train_dataset = Dataset('train').make_batch(batch_size, train_files)
train_steps = -(-mt.get_num_records(train_files) // batch_size)
steps = -(-mt.get_num_records(eval_files) // batch_size)
FLAGS.NUM_CLASSES = NUM_CLASSES

['../input/quarter/tfrecords/train/1/1.3334.tfrec', '../input/quarter/tfrecords/train/1/11.3333.tfrec', '../input/quarter/tfrecords/train/1/21.3333.tfrec']


In [12]:
def eval(model):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = get_metrics()
  model.compile('sgd', loss_fn, metrics=metrics)
  res = model.evaluate(eval_dataset, steps=steps, return_dict=True)
#   gezi.pprint_dict(res)
  cm = mt.distributed.sum_merge(metrics[0].get_cm())
  infos = util.get_infos_from_cm(cm, CLASSES)
  res.update(infos)
  return res

In [13]:
res = eval(model)

625/625 [==============================] - 21s 34ms/step - loss: 2.4594 - FWIoU: 0.1953 - MIoU: 0.0692 - ACC/pixel: 0.3730 - ACC/class: 0.1411
water                |true:0.0273 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.5907|
track_road           |true:0.0767 pred:0.0000 acc:0.1784 recall:0.0000 iou:0.0000 fwiou:-0.1507|
build                |true:0.1212 pred:0.0004 acc:0.9179 recall:0.0031 iou:0.0031 fwiou:-0.8129|
track_airport        |true:0.0001 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9991|
other_park           |true:0.0665 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.0029|
other_playground     |true:0.0033 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9511|
arable_natural       |true:0.2266 pred:0.3518 acc:0.5787 recall:0.8986 iou:0.5433 fwiou:-0.5523|
arable_greenhouse    |true:0.0129 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.8064|
grass_natural        |true:0.1275 pred:0.4832 acc:0.1911 recall:0.7242 iou:0.1781 fwiou:-0.5718|
gras